# Final Project - Asian Option Pricing

**Université Paris Dauphine - Master 1  
Monte Carlo Methods (2023-2024)**

## Context

We consider a risky asset $S$ defined by
$$
S_t = S_0 \exp\left(\left(r - \frac{\sigma^2}{2}\right)t + \sigma W_t\right),
$$
where $S_0 = 1$, $r = 0$, and $W_t$ is a standard Brownian motion.

We define a time grid
$$
t_k = \frac{T}{n} k,\quad k = 0,1,\dots,n,
$$
with $T = 1$. The arithmetic average of prices is given by
$$
A_T^n = \frac{1}{n}\sum_{i=1}^{n} S_{t_i}.
$$

The Asian option payoff is defined as
$$
G = \bigl(A_T^n - K\bigr)^+,
$$
with $K = 1$.  
The goal is to compute
$$
E[G] = E\left[\bigl(A_T^n - K\bigr)^+\right]
$$
using Monte Carlo methods.

## Tasks

1. **Simulation and Monte Carlo Estimation**  
   - Simulate Brownian motion trajectories on the discrete grid.  
   - Simulate the trajectories of $S_t$.  
   - Estimate $E[G]$.

2. **Antithetic Variates Method**  
   - Use antithetic variates to reduce variance.

3. **Control Variates Method**  
   - Use the control variate
     $$
     \left(\left(\prod_{i=1}^{n} S_{t_i}\right)^{1/n} - K\right)^+
     $$
     to improve the estimation.

4. **Importance Sampling**  
   - Express $G = g(Z)$ where $Z \sim \mathcal{N}(0,I_n)$.  
   - Show that for any vector $\mu \in \mathbb{R}^n$,
     $$
     E[G] = E\left[g(Z+\mu)\exp\left(-\langle \mu,Z\rangle-\frac{\|\mu\|^2}{2}\right)\right].
     $$
   - Following [1], determine an approximate vector $\hat{\mu}$ via the bifurcation method.

5. **(Bonus) Stratification**  
   - Propose a stratification method to further reduce variance.

The following code cell provides a Python implementation of these methods.


In [3]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import bisect

# Option and model parameters
S0 = 1.0      # Initial price
r = 0.0       # Interest rate (zero)
sigma = 0.25  # Volatility
T = 1.0       # Maturity
K = 1.0       # Strike price
n = 50        # Number of time steps
dt = T / n    # Time step
M = 10000     # Number of Monte Carlo simulations

#####################################
# 1. Simulation and Standard Monte Carlo
#####################################
def simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=False):
    """
    Simulate asset price paths S_t on the discrete grid.
    If antithetic=True, use antithetic variates for variance reduction.
    Returns an array of shape (M, n+1) with the simulated paths.
    """
    if not antithetic:
        # Standard simulation
        dW = np.sqrt(dt) * np.random.randn(M, n)
        S = np.zeros((M, n+1))
        S[:, 0] = S0
        for i in range(n):
            S[:, i+1] = S[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, i])
        return S
    else:
        # Antithetic variates simulation: M/2 pairs
        M_half = M // 2
        dW = np.sqrt(dt) * np.random.randn(M_half, n)
        dW_anti = -dW  # opposite trajectories
        S1 = np.zeros((M_half, n+1))
        S2 = np.zeros((M_half, n+1))
        S1[:, 0] = S0
        S2[:, 0] = S0
        for i in range(n):
            S1[:, i+1] = S1[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, i])
            S2[:, i+1] = S2[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW_anti[:, i])
        S = np.concatenate([S1, S2], axis=0)
        return S

def asian_option_payoff(S, K):
    """
    Compute the payoff of the Asian option for each path.
    The average is calculated over time steps t_1 to t_n.
    """
    A = np.mean(S[:, 1:], axis=1)
    payoff = np.maximum(A - K, 0)
    return payoff

def monte_carlo_price(S, K):
    """
    Estimate the option price using the Monte Carlo method.
    Returns the mean payoff and the standard error.
    """
    payoff = asian_option_payoff(S, K)
    price = np.mean(payoff)
    std_error = np.std(payoff) / np.sqrt(len(payoff))
    return price, std_error

# Standard Monte Carlo simulation
S_paths = simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=False)
price_mc, se_mc = monte_carlo_price(S_paths, K)
print("Standard Monte Carlo:")
print("Price: {:.4f}, Standard Error: {:.4f}".format(price_mc, se_mc))

#####################################
# 2. Variance Reduction with Antithetic Variates
#####################################
S_paths_anti = simulate_asset_paths(M, n, S0, r, sigma, dt, antithetic=True)
price_anti, se_anti = monte_carlo_price(S_paths_anti, K)
print("\nAntithetic Variates Method:")
print("Price: {:.4f}, Standard Error: {:.4f}".format(price_anti, se_anti))

#####################################
# 3. Variance Reduction with Control Variates
#####################################
def control_variate_payoff(S, K):
    """
    Compute the payoff of the control variate based on the geometric average:
    $$\left(\prod_{i=1}^{n} S_{t_i}\right)^{1/n} - K.$$
    """
    geo_mean = np.exp(np.mean(np.log(S[:, 1:]), axis=1))
    payoff_cv = np.maximum(geo_mean - K, 0)
    return payoff_cv

# Compute payoffs
payoff = asian_option_payoff(S_paths, K)
control_payoff = control_variate_payoff(S_paths, K)

# Exact price of the geometric Asian option (analytical approximation)
def geometric_asian_option_price(S0, K, r, sigma, T, n):
    """
    Compute the exact price of the geometric Asian option.
    """
    dt = T/n
    sigma_sq_geo = sigma**2 * ((n+1)*(2*n+1)) / (6*n**2)
    sigma_geo = np.sqrt(sigma_sq_geo)
    mu_geo = (r - 0.5*sigma**2)*(T + dt)/2 + 0.5*sigma_sq_geo
    d1 = (np.log(S0/K) + mu_geo) / sigma_geo
    d2 = d1 - sigma_geo
    price_geo = S0 * np.exp(mu_geo) * norm.cdf(d1) - K * norm.cdf(d2)
    return price_geo

price_geo_exact = geometric_asian_option_price(S0, K, r, sigma, T, n)

# Control variate estimation
cov_matrix = np.cov(payoff, control_payoff)
beta = - cov_matrix[0,1] / cov_matrix[1,1]
adjusted_payoff = payoff + beta * (control_payoff - price_geo_exact)
price_cv = np.mean(adjusted_payoff)
se_cv = np.std(adjusted_payoff) / np.sqrt(len(adjusted_payoff))
print("\nControl Variates Method:")
print("Price: {:.4f}, Standard Error: {:.4f}".format(price_cv, se_cv))

#####################################
# 4. Importance Sampling
#####################################
def compute_mu_hat(y, S0, sigma, dt, n, K):
    """
    For a given y, compute the sequence z(y) and S(y) and return:
    $$ \frac{1}{n}\sum_{j=1}^{n} S_j(y) - K - y. $$
    """
    z = np.zeros(n)
    S_vec = np.zeros(n)
    z[0] = sigma * np.sqrt(dt) * (y + K) / y
    t1 = dt
    S_vec[0] = S0 * np.exp(-0.5*sigma**2 * t1 + sigma * np.sqrt(dt) * z[0])
    for j in range(1, n):
        z[j] = z[j-1] - sigma * np.sqrt(dt) * S_vec[j-1] / (n * y)
        t_j = (j+1) * dt
        S_vec[j] = S0 * np.exp(-0.5*sigma**2 * t_j + sigma * np.sqrt(dt) * np.sum(z[:j+1]))
    avg_S = np.mean(S_vec)
    return avg_S - K - y

# Find y_hat using bisection method
y_lower = 1e-4
y_upper = 1.0
while compute_mu_hat(y_lower, S0, sigma, dt, n, K) * compute_mu_hat(y_upper, S0, sigma, dt, n, K) > 0:
    y_upper *= 2

y_hat = bisect(lambda y: compute_mu_hat(y, S0, sigma, dt, n, K), y_lower, y_upper, xtol=1e-4)
print("\nImportance Sampling:")
print("y_hat =", y_hat)

# Compute mu_hat from y_hat
def compute_mu_from_y(y, S0, sigma, dt, n):
    z = np.zeros(n)
    z[0] = sigma * np.sqrt(dt) * (y + K) / y
    for j in range(1, n):
        S_j = S0 * np.exp(-0.5*sigma**2 * (j*dt) + sigma * np.sqrt(dt) * np.sum(z[:j]))
        z[j] = z[j-1] - sigma * np.sqrt(dt) * S_j / (n * y)
    return z

mu_hat = compute_mu_from_y(y_hat, S0, sigma, dt, n)
print("mu_hat =", mu_hat)

# Simulate paths using Importance Sampling
def simulate_paths_importance_sampling(M, n, S0, r, sigma, dt, mu):
    Z_shifted = np.random.randn(M, n) + mu
    S = np.zeros((M, n+1))
    S[:, 0] = S0
    for i in range(n):
        S[:, i+1] = S[:, i] * np.exp((r - 0.5 * sigma**2)*dt + sigma * np.sqrt(dt) * Z_shifted[:, i])
    Z_original = Z_shifted - mu
    return S, Z_original

S_paths_is, Z_samples = simulate_paths_importance_sampling(M, n, S0, r, sigma, dt, mu_hat)
weights = np.exp(-np.sum(mu_hat * Z_samples, axis=1) - 0.5 * np.sum(mu_hat**2))
payoff_is = asian_option_payoff(S_paths_is, K)
weighted_payoff = payoff_is * weights
price_is = np.mean(weighted_payoff)
se_is = np.std(weighted_payoff) / np.sqrt(M)
print("Price with Importance Sampling: {:.4f}, Standard Error: {:.4f}".format(price_is, se_is))

#####################################
# 5. (Bonus) Stratification
#####################################
def stratified_sampling(M, n, S0, r, sigma, dt, K, strata=10):
    M_per_stratum = M // strata
    S_all = []
    for i in range(strata):
        a = norm.ppf(i / strata)
        b = norm.ppf((i+1) / strata)
        Z1_val = norm.ppf((i + 0.5) / strata)
        dW_remaining = np.sqrt(dt) * np.random.randn(M_per_stratum, n-1)
        dW_full = np.hstack([Z1_val * np.ones((M_per_stratum, 1)), dW_remaining])
        S = np.zeros((M_per_stratum, n+1))
        S[:, 0] = S0
        for j in range(n):
            S[:, j+1] = S[:, j] * np.exp((r - 0.5 * sigma**2)*dt + sigma * dW_full[:, j])
        S_all.append(S)
    S_all = np.vstack(S_all)
    payoff_all = asian_option_payoff(S_all, K)
    price_strat = np.mean(payoff_all)
    se_strat = np.std(payoff_all) / np.sqrt(len(payoff_all))
    return price_strat, se_strat

price_strat, se_strat = stratified_sampling(M, n, S0, r, sigma, dt, K, strata=10)
print("\nStratification:")
print("Price: {:.4f}, Standard Error: {:.4f}".format(price_strat, se_strat))


Standard Monte Carlo:
Price: 0.0586, Standard Error: 0.0010

Antithetic Variates Method:
Price: 0.0575, Standard Error: 0.0009

Control Variates Method:
Price: 0.0582, Standard Error: 0.0000

Importance Sampling:
y_hat = 0.15834833374023435
mu_hat = [0.25863106 0.25412735 0.24958582 0.2450069  0.24039104 0.23573868
 0.23105032 0.22632645 0.22156761 0.21677434 0.21194721 0.20708682
 0.20219376 0.19726868 0.19231223 0.18732508 0.18230793 0.17726149
 0.1721865  0.16708371 0.16195389 0.15679784 0.15161636 0.1464103
 0.14118048 0.13592777 0.13065307 0.12535724 0.12004122 0.11470593
 0.1093523  0.10398129 0.09859387 0.09319101 0.08777371 0.08234297
 0.0768998  0.07144522 0.06598025 0.06050595 0.05502336 0.04953351
 0.04403749 0.03853633 0.03303112 0.02752292 0.02201281 0.01650185
 0.01099111 0.00548168]
Price with Importance Sampling: 0.0582, Standard Error: 0.0003

Stratification:
Price: 0.1261, Standard Error: 0.0020


<>:86: SyntaxWarning: invalid escape sequence '\l'
<>:127: SyntaxWarning: invalid escape sequence '\s'
<>:86: SyntaxWarning: invalid escape sequence '\l'
<>:127: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ugo11\AppData\Local\Temp\ipykernel_26836\3104930366.py:86: SyntaxWarning: invalid escape sequence '\l'
  """
C:\Users\ugo11\AppData\Local\Temp\ipykernel_26836\3104930366.py:127: SyntaxWarning: invalid escape sequence '\s'
  """
